# Car System Agentic AI - Template

**Project:** Agentic AI example  
**Author:** Klaus  
**License:** MIT License

---

This notebook demonstrates the car diagnostic system using LangGraph with multiple specialized nodes for input validation, reasoning, and output processing.


# Imports

In [ ]:
exec(open("setup.py").read())

from notebooks import *
from langgraph.constants import START
from langgraph.graph import StateGraph
from src.data_models.graph_state import CarSystemState
from src.tools.car import get_car_status
from src.tools.calculations import is_trip_possible
from src.tools.travel import recommend_locations
from src.tools.weather import get_predicted_weather
from src.tools.agent_registry import list_registered_agents, invoke_agent
from src.utils.prompt_loader import get_prompt
from src.utils.agent_card_loader import load_agent_cards_from_file
from src.services.agent_registry import AgentRegistry
from src.utils.test_graph import test_graph
from pathlib import Path

# Load Prompts

In [ ]:
# Load all prompts
output_guard_rail_prompt = get_prompt("output_guard_rail")
input_guard_rail_prompt = get_prompt("input_guard_rail")
reasoning_node_prompt = get_prompt("reasoning_node")
car_central_prompt = get_prompt("car_central")
trip_planner_prompt = get_prompt("trip_planner")

In [ ]:
# Load and register AgentCards from JSON (pt-BR)
cards_path = Path.cwd().parent / "data" / "agent_cards.json"
agent_cards = load_agent_cards_from_file(cards_path)
AgentRegistry.clear()

# Extract specific cards by name
car_card = next(c for c in agent_cards if c.name == "AgenteDiagnosticoCarro")
trip_card = next(c for c in agent_cards if c.name == "AgentePlanejadorViagem")

# Definitions

## Agents

In [ ]:
# Specialized agents

car_agent_model = models.GeminiModel(
    model="gemini-2.5-flash",
    prompt=car_central_prompt,
    agent_card=car_card,
    tools=[get_car_status],
)

trip_planner_agent_model = models.GeminiModel(
    model="gemini-2.5-flash",
    prompt=trip_planner_prompt,
    agent_card=trip_card,
    tools=[recommend_locations, get_predicted_weather],
)

# Register cards with their models
AgentRegistry.register(car_card, car_agent_model)
AgentRegistry.register(trip_card, trip_planner_agent_model)

# Graph agents

# Input guard rail agent
input_guard_rail_agent = models.GeminiModel(
    model="gemini-2.5-flash", prompt=input_guard_rail_prompt
)

# Reasoning agent (orchestration + quick feasibility)
reasoning_agent = models.GeminiModel(
    model="gemini-2.5-flash",
    prompt=reasoning_node_prompt,
    tools=[list_registered_agents, invoke_agent, is_trip_possible],
)

# Output guard rail agent
output_guard_rail_agent = models.GeminiModel(
    model="gemini-2.5-flash", prompt=output_guard_rail_prompt
)

## Nodes

In [ ]:
# node definition
ENTRYPOINT = START
INPUT_GUARD_RAIL = "input_guard_rail"
REASONING_NODE = "reasoning_node"
OUTPUT_GUARD_RAIL = "output_guard_rail"
EXIT_ZONE = "end"

input_guard_rail = nodes.InputGuardRail(
    routing_options={"next_node": REASONING_NODE, "end": OUTPUT_GUARD_RAIL},
    model=input_guard_rail_agent,
)

reasoning_node = nodes.ReasoningNode(
    routing_options={"next_node": OUTPUT_GUARD_RAIL, "end": OUTPUT_GUARD_RAIL},
    model=reasoning_agent,
)

output_guard_rail = nodes.OutputGuardRail(
    routing_options={"end": EXIT_ZONE},
    model=output_guard_rail_agent,
)

## Graph Creation

In [ ]:
# Create the graph with state_schema parameter
workflow = StateGraph(state_schema=CarSystemState)

# Add nodes to the graph
workflow.add_node(INPUT_GUARD_RAIL, input_guard_rail)
workflow.add_node(REASONING_NODE, reasoning_node)
workflow.add_node(OUTPUT_GUARD_RAIL, output_guard_rail)

# Define the flow with conditional routing
# Os nodes usam Command para determinar o próximo node baseado no estado
workflow.add_edge(START, "input_guard_rail")
# workflow.add_edge(EXIT_ZONE, lambda state: Command(goto=END))

app = workflow.compile()

# Test Graph Execution

In [ ]:
human_query = "Qual local voce recomenda para eu viajar?"
sync_result = test_graph(app, human_query)

In [ ]:
human_query = "Quero viajar 300 km amanhã. Com o combustível que tenho dá para ir sem reabastecer?"
sync_result = test_graph(app, human_query)